In [ ]:
%matplotlib inline
import os
import sys
import gc
import time
from tqdm.notebook import tqdm

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from commons.configuration_manager import ConfigurationManager
from src.utilities.transformer import Transformer
from src.learning.training.generator import Generator, GenFiles
from src.learning.models import create_cnn, create_mlp, create_multi_model

In [ ]:
def plot_stuff(title, plot_elems, figsize=(18, 10)):
    fig=plt.figure(figsize=figsize)
    plt.title(title)
    #plt.ylabel('dunno')
    plt.xlabel('Epoch')
    x = np.arange(0, len(plot_elems[0]['data']), 1)
    
    for plot_elem in plot_elems:
        plt.errorbar(x, plot_elem['data'], yerr=plot_elem['error'], label=plot_elem['label'], alpha=plot_elem['alpha'], fmt='-o', capsize=5)

    plt.grid(axis='x')
    plt.legend(loc='best', prop={'size': 15})
    plt.show()
    plt.savefig('./' + title + '.png')

In [ ]:
config_manager = ConfigurationManager()
config = config_manager.config

memory_variants = [(1, 1), (4, 1), (4, 4), (8, 1), (8, 4)]
memory_variants = [(1, 1), (4, 1)]
spec_memory = memory_variants[1]

model_path = '../../training/models/'

In [ ]:
epochs = 10
batch_size = 64
verbose = 1

losses = []
val_losses = []
    
for memory in tqdm(memory_variants):
    result_desc = 'n{}_m{}'.format(*memory)
    tqdm.write(result_desc)
    
    generator = Generator(config, memory_tuple=memory, base_path='../../training/', batch_size=batch_size, column_mode='steer', separate_files=True)
    frame_shape, numeric_shape, diff_shape = generator.get_shapes()

    mlp = create_mlp(input_shape=numeric_shape)
    cnn = create_cnn(input_shape=frame_shape)
    multi = create_multi_model(mlp, cnn, output_shape=diff_shape)

    hist = multi.fit(generator.generate(data='train'),
                     steps_per_epoch=generator.train_batch_count,
                     validation_data=generator.generate(data='test'),
                     validation_steps=generator.test_batch_count,
                     epochs=epochs, verbose=verbose)

    model_file_prefix = 'model_n{}_m{}'.format(*memory)
    model_file_suffix = '_{}.{}'

    count = len([fn for fn in os.listdir(model_path) if fn.startswith(model_file_prefix) and fn.endswith('.h5')])
    plot_model(multi, to_file=model_path + model_file_prefix + model_file_suffix.format(count + 1, 'png'))
    multi.save(model_path + model_file_prefix + model_file_suffix.format(count + 1, 'h5'))
    
    current_loss = hist.history['loss']
    current_val_loss = hist.history['val_loss'] 
    
    losses.append(current_loss)
    val_losses.append(current_val_loss)
    
    tqdm.write("Loss per epoch: {}".format(current_loss))
    tqdm.write("Validation loss per epoch: {}".format(current_val_loss))
    
    gc.collect()

In [ ]:
loss_data = []
val_loss_data = []

for i in range(0, len(losses)):
    val_loss_data.append({'data': val_losses[i], 'label': key, 'alpha': 1.0})
    loss_data.append({'data': losses[i], 'label': key, 'alpha': 1.0})
        
plot_stuff("val losses", val_loss_data, figsize=(10, 14))
plot_stuff("losses", loss_data, figsize=(10, 14))

In [ ]:
import tensorflow as tf

generator = Generator(config, memory_tuple=spec_memory, base_path='../../training/', column_mode='steer', separate_files=True)
frame_shape, numeric_shape, diff_shape = generator.get_shapes()

super_shapes = ((frame_shape, numeric_shape), diff_shape)
print(super_shapes)
train_dataset = tf.data.Dataset.from_generator(generator.generate_single_train, output_types=(np.float32, np.float32))
train_dataset = train_dataset.batch(batch_size=64)
train_dataset = train_dataset.shuffle(buffer_size=1000)

val_dataset = tf.data.Dataset.from_generator(generator.generate_single_test, (np.float32, np.float32))
val_dataset = val_dataset.batch(batch_size=64)
val_dataset = val_dataset.shuffle(buffer_size=1000)

mlp = create_mlp(input_shape=numeric_shape)
cnn = create_cnn(input_shape=frame_shape)
multi = create_multi_model(mlp, cnn, output_shape=diff_shape)

hist = multi.fit_generator(train_dataset,
                 steps_per_epoch=generator.train_batch_count,
                 validation_data=val_dataset,
                 validation_steps=generator.test_batch_count,
                 epochs=10, verbose=1)